In [177]:
# Tic-tac toe 
# [TODO] create an input value that can be entered into the game 
# create a separate Agent Class 
# 

import numpy as np
import random as re

class ticTacToe:
    
    def __init__(self, turn = "o"):
        # start with users playing "o"
        self.board = np.zeros((3,3))
        self.free_positions = set([ (0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)])
        # self.input_position = input('Input your position as x,y with positions values from 0 to 2')
        self.game_over = False
        # start with user playing
        # self.turn = turn
    
    def printBoard(self):
        print('New play:')
        for row in self.board:
            print("%s | %s | %s" % (self.convertToXO(row[0]), 
                                    self.convertToXO(row[1]), 
                                    self.convertToXO(row[2])))

    def convertToXO(self, num):
        if num==0:
            return "-"
        if num==-1:
            return "o"
        if num==1:
            return "x"
            
    def makeMove(self, position, turn):
        pos1 = position[0]
        pos2 = position[1]
        if pos1>=len(self.board):
            print('Illegal move, try again')
            return False
        elif pos2>=len(self.board[0]):
            print('Illegal move, try again')
            return False
        elif self.board[pos1][pos2]==0:
            self.board[pos1][pos2] = turn
            self.free_positions.remove(position)
            self.printBoard()
            return True
        else:
            print('Illegal move, try again')
            return False
    
            
    def userPlayMove(self, agent):
        self.input_position = input('Input your position as x,y with positions values from 0 to 2:')
        position = self.checkInputAndReturn(self.input_position)
        is_legit = self.makeMove(position, -1)
        if is_legit:
            self.checkWinner(self.board)
            print('Agent thinking....')
            self.board, self.free_positions, self.game_over = agent.agentMakeMove(
                self.board, self.free_positions, self.game_over)
            self.printBoard()
            self.checkWinner(self.board)
        else:
            print('Try again not legit')
        
    def checkInputAndReturn(self, input_position):
        # check that input and return value
        valid_input1 = input_position[0] in ('0','1','2')
        valid_input2 = input_position[1] in ('0','1','2')
        if not (valid_input1 or valid_input2):
            print('Not Valid, try again')
            self.input_position = input('Input your position as x,y with positions values from 0 to 2:')
        else:
            position = (int(input_position[0]), int(input_position[2]))
            return position
        
    def checkWinner(self, board):
        row_sums = np.sum(board, axis=0)
        col_sums = np.sum(board, axis=1)
        diag_sum1 = board[0,0] + board[1,1] + board[2,2]  
        diag_sum2 = board[0,2] + board[1,1] + board[2,0] 
        max_row_sums = max(row_sums)
        max_col_sums = max(col_sums)
        max_sum = max(max_row_sums, max_col_sums, diag_sum1, diag_sum2)
        
        min_row_sums = min(row_sums)
        min_col_sums = min(col_sums)
        min_sum = min(min_row_sums, min_col_sums, diag_sum1, diag_sum2)
        if max_sum==len(board):
            print('Agent wins!!')
            self.game_over = True
            return True
        elif min_sum==-len(board):
            print('User wins!!')
            self.game_over = True
            return True
        else:
            return False

        
class Agent:
    
    def __init__(self, board, free_positions):
        self.board = board
        self.free_positions = free_positions

    def agentMakeMove(self, board, free_positions, game_over):
        # random picks a move from free positions
        if self.isFull():
            print('Full board, tied game over!')
            return board, free_positions, True
        elif not game_over:
            # [TODO] add smart logic here
            agent_move = self.agentMakeDecision(board, free_positions)
            self.makeMove(agent_move, 1)
        return  self.board, self.free_positions, False

    def isFull(self):
        return len(self.free_positions) == 0
            
    def makeMove(self, position, turn):
        pos1 = position[0]
        pos2 = position[1]
        self.board[pos1][pos2] = turn
        self.free_positions.remove(position)
        
    def agentMakeDecision(self, board = None, free_positions = None):
        if board is not None:
            self.board = board
        if free_positions is not None:
            self.free_positions = free_positions
            
        # if it can in the next round it does
        # check every position available and if the row already or col already has two agent
        # then play
        diag_set1 = set([(0,0),(1,1),(2,2)])
        diag_set2 = set([(0,2),(1,1),(0,2)])
        diag1 = [self.board[0,0], self.board[1,1], self.board[2,2]]
        diag2 = [self.board[0,2], self.board[1,1], self.board[0,2]]
        
        for pos in self.free_positions:
            board_row = self.board[pos[0],:]
            board_col = self.board[:,pos[1]]
            if self.checkWinnerMove(pos, board_row, board_col, diag1, diag2):
                return pos
            
        # otherwise if no position exists 
        # check every position available and if the row or col already has two user pieces, play 
        for pos in self.free_positions:
            board_row = self.board[pos[0],:]
            board_col = self.board[:,pos[1]]
            if self.checkBlockMove(pos, board_row, board_col, diag1, diag2):
                return pos

        # otherwise if no position exists
        # check every position available and if the row or col has one agent piece
        # and no user piece, then play 
        for pos in self.free_positions:
            board_row = self.board[pos[0],:]
            board_col = self.board[:,pos[1]]
            if self.checkStrategicMove(pos, board_row, board_col, diag1, diag2):
                return pos
            
        # if no other strategic move, randomly sample one
        return re.sample(self.free_positions,1)[0]

    
    def checkWinnerMove(self, pos, board_row, board_col, diag1, diag2):
        diag_set1 = set([(0,0),(1,1),(2,2)])
        diag_set2 = set([(0,2),(1,1),(0,2)])

        is_row_winner = np.sum(board_row) == 2 and min(board_row)==0
        is_col_winner = np.sum(board_col) == 2 and min(board_col)==0
        is_diag_winner1 = pos in diag_set1 and  np.sum(diag1)==2 and min(diag1)==0
        is_diag_winner2 = pos in diag_set2 and  np.sum(diag2)==2 and min(diag2)==0

        return max(is_row_winner, is_col_winner, is_diag_winner1, is_diag_winner2)

    def checkBlockMove(self, pos, board_row, board_col, diag1, diag2):
        diag_set1 = set([(0,0),(1,1),(2,2)])
        diag_set2 = set([(0,2),(1,1),(0,2)])

        is_row_block = np.sum(board_row) == -2 and max(board_row)==0
        is_col_block = np.sum(board_col) == -2 and max(board_col)==0
        is_diag_block1 = pos in diag_set1 and  np.sum(diag1)==-2 and max(diag1)==0
        is_diag_block2 = pos in diag_set2 and  np.sum(diag2)==-2 and max(diag2)==0

        return max(is_row_block, is_col_block, is_diag_block1, is_diag_block2)

    def checkStrategicMove(self, pos, board_row, board_col, diag1, diag2):
        diag_set1 = set([(0,0),(1,1),(2,2)])
        diag_set2 = set([(0,2),(1,1),(0,2)])

        is_row_strat = np.sum(board_row) == 1 and min(board_row)==0
        is_col_strat = np.sum(board_col) == 1 and min(board_col)==0
        is_diag_strat1 = pos in diag_set1 and  np.sum(diag1)==1 and min(diag1)==0
        is_diag_strat2 = pos in diag_set2 and  np.sum(diag2)==1 and min(diag2)==0

        return max(is_row_strat, is_col_strat, is_diag_strat1, is_diag_strat2)


game = ticTacToe()
agent = Agent(game.board, game.free_positions)
game.userPlayMove(agent)



New play:
- | - | -
- | - | -
- | - | -
Input your position as x,y with positions values from 0 to 2:1,1
New play:
- | - | -
- | o | -
- | - | -
agent thinking....
New play:
- | - | -
x | o | -
- | - | -


In [178]:
game.userPlayMove(agent)


Input your position as x,y with positions values from 0 to 2:0,0
New play:
o | - | -
x | o | -
- | - | -
agent thinking....
New play:
o | - | -
x | o | -
- | - | x


In [180]:
game.userPlayMove(agent)

Input your position as x,y with positions values from 0 to 2:0,2
New play:
o | o | o
x | o | -
- | x | x
User wins
agent thinking....
New play:
o | o | o
x | o | -
- | x | x
User wins


In [133]:
position = input('enter x,y with comma-delimit')

enter x,y with comma-delimit0,1


In [134]:
position

'0,1'